## 경상북도 의회 의안 목록 수집

In [39]:
page_no = 1
def get_page_url(page_no):
    url = "http://council.gb.go.kr/kr/bill/contents"
    url = f"{url}?reform=list&page={page_no}&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003"
    url = f"{url}&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch=&cl_sch=&th_sch=12&sess_sch="
    url = f"{url}&ord_sch=&cmt_sch=&mem_sch=&sess_sth=&sess_eth="
    return url

In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [41]:
url = get_page_url(page_no=1)
print(url)

http://council.gb.go.kr/kr/bill/contents?reform=list&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch=&cl_sch=&th_sch=12&sess_sch=&ord_sch=&cmt_sch=&mem_sch=&sess_sth=&sess_eth=


In [47]:
def find_one_page(page_no):
    response = requests.get(get_page_url(page_no))
    html = bs(response.text)
    # #search_result > table > tbody > tr:nth-child(1) > td.con > a
    a_href = [a["href"] for a in html.select("td.con > a")]
    table = pd.read_html(response.text)[0]
    table["내용링크"] = a_href
    return table

In [48]:
page_tables = [find_one_page(page_no) for page_no in range(1, 9)]

In [49]:
df = pd.concat(page_tables)

In [51]:
df[df.duplicated()]

,의안번호,의안명,제안자,소관위원회,발의일,상태,의결일,내용링크


In [52]:
file_name = "경상북도의안.csv"
df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,의안번호,의안명,제안자,소관위원회,발의일,상태,의결일,내용링크
0,205,경상북도교육청 다자녀 학생 교육비 지원에 관한 조례 일부개정조례안,의원,교육위원회,2023-02-24,처리완료,2023-03-22,?reform=view&uid=5A40BDF4228EFF81E975DF436D9F8...
1,204,경상북도교육청 학교폭력예방 및 대책에 관한 조례 일부개정조례안,의원,교육위원회,2023-02-24,처리완료,2023-03-22,?reform=view&uid=F161A1CBE65C4C05D46123037FEE5...
2,203,경상북도 안전취약계층에 대한 안전 환경 지원 조례안,의원,건설소방위원회,2023-02-24,처리완료,2023-03-22,?reform=view&uid=458FD0B3F12FCDC04F14B259867AC...
3,202,경상북도 건축 조례 일부개정조례안,의원,건설소방위원회,2023-02-24,처리완료,2023-03-22,?reform=view&uid=F1775B299D48E0A3A76E2FC9E9AF5...
4,201,경상북도 해양폐기물 관리에 관한 조례 일부개정조례안,의원,농수산위원회,2023-02-24,처리완료,2023-03-22,?reform=view&uid=C5F67C31EACB8F3F67812C49EA4F5...
...,...,...,...,...,...,...,...,...
72,32,경상북도 빈집 및 소규모주택 정비 조례 일부개정조례안,의원,건설소방위원회,2022-08-11,처리완료,2022-09-05,?reform=view&uid=510B0B09EE74A11A0C09B3D69AA1C...
73,28,경상북도교육청 민원 처리 담당자 보호 및 지원 조례안,의원,교육위원회,2022-08-10,처리완료,2022-09-05,?reform=view&uid=63F2439F78F6FB6BF6E9C67069B68...
74,27,경상북도교육청 주민참여 예산제 운영 조례 전부개정조례안,의원,교육위원회,2022-08-10,처리완료,2022-09-05,?reform=view&uid=95D21E5EF3F707C9065E9583F20D5...
75,26,경상북도교육청 학업중단 예방 및 대안교육 지원 조례 일부개정조례안,의원,교육위원회,2022-08-10,처리완료,2022-09-05,?reform=view&uid=EF0D0CCBE07D54808CA23B04DEEF5...


In [54]:
content_base_url = "http://council.gb.go.kr/kr/bill/contents"

In [99]:
def find_content(sub_url):
    content_base_url = "http://council.gb.go.kr/kr/bill/contents" 
    url = content_base_url + sub_url
    
    response = requests.get(url)
    soup = bs(response.text)
    
    # a.bill_file.view_book.btn_bill
    a_tags = soup.select("a.bill_file.view_book.btn_bill")
    a_result = [{a.get_text() : a["title"], a.get_text()+"URL" : a["href"]} for a in a_tags]


    # a.bill_file.view_book.btn
    a_view_tags = soup.select("a.bill_file.view_book.btn")
    a_view_result = [{a.get_text() : a["title"]+str(i), a.get_text()+"URL" : a["href"]} for i, a in enumerate(a_view_tags)]

    return a_result, a_view_result

In [101]:
a, b = find_content(sub_url=df.iloc[0, -1])
print(a, b)

[{'의안원문': '205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개정조례안.hwp', '의안원문URL': '/kr/bill/download?uid=91EC2C452C29B56E73D1ABDD63D0A855&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='}, {'심사보고서': '205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부개정조례안(최종).pdf', '심사보고서URL': '/kr/bill/download?uid=9B0DABB4724A19390DC53CD38A2ECB0C&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='}] [{'바로보기': '205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개정조례안.hwp0', '바로보기URL': '/kr/bill/viewdownload?uid=91EC2C452C29B56E73D1ABDD63D0A855&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='}, {'바로보기': '205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부개정조례안(최종).pdf1', '바로보기URL': '/kr/bill/viewdownload?uid=9B0DABB4724A19390DC53CD38A2ECB0C&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='}]


In [98]:
pd.DataFrame(cont)

,0,1
0,{'의안원문': '205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개...,{'심사보고서': '205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부...
1,{'바로보기': '205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개...,{'바로보기': '205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부개...


In [61]:
response = requests.get(url)
soup = bs(response.text)

In [91]:
# #sub_board > div > table:nth-child(4) > tbody > tr > td:nth-child(5) > ul > li > a.bill_file.view_book.btn_bill
a_tags = soup.select("a.bill_file.view_book.btn_bill")
result = [{a.get_text() : a["title"], a.get_text()+"URL" : a["href"]} for a in a_tags]

result
# a_tags[0]["href"], a_tags[0]["title"]

[{'의안원문': '205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개정조례안.hwp',
  '의안원문URL': '/kr/bill/download?uid=91EC2C452C29B56E73D1ABDD63D0A855&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='},
 {'심사보고서': '205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부개정조례안(최종).pdf',
  '심사보고서URL': '/kr/bill/download?uid=9B0DABB4724A19390DC53CD38A2ECB0C&page=1&flag=all&keyword=&search_code=&bill_sch=C001&prop_sch=A003&proc_sch=&sdate=&edate=&pcode_sch=&acode_sch='}]

In [86]:
a_tags[0].get_text()

'의안원문'

In [83]:
a_tags[0]["title"]

'205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개정조례안.hwp'

In [66]:
#sub_board > div > table:nth-child(4) > tbody > tr > td:nth-child(5) > ul > li > a.bill_file.view_book.btn
a_view_tags = soup.select("a.bill_file.view_book.btn")
a_view_tags

[<a class="bill_file view_book btn" href="/kr/bill/viewdownload?uid=91EC2C452C29B56E73D1ABDD63D0A855&amp;page=1&amp;flag=all&amp;keyword=&amp;search_code=&amp;bill_sch=C001&amp;prop_sch=A003&amp;proc_sch=&amp;sdate=&amp;edate=&amp;pcode_sch=&amp;acode_sch=" target="_blank" title="205-경상북도교육청_다자녀_학생_교육비_지원에_관한_조례_일부개정조례안.hwp"><span>바로</span>보기</a>,
 <a class="bill_file view_book btn" href="/kr/bill/viewdownload?uid=9B0DABB4724A19390DC53CD38A2ECB0C&amp;page=1&amp;flag=all&amp;keyword=&amp;search_code=&amp;bill_sch=C001&amp;prop_sch=A003&amp;proc_sch=&amp;sdate=&amp;edate=&amp;pcode_sch=&amp;acode_sch=" target="_blank" title="205_(심사보고)경상북도교육청_다자녀학생교육비지원에관한조례일부개정조례안(최종).pdf"><span>바로</span>보기</a>]